In [99]:
i=-1

In [1]:
%run gesLmUtilsLib.ipynb

In [153]:
class Generator2():
    
    def __init__(self, BasicGesturePath="garbage/GestureProject/RawGestures/RelocatedRawGes10R.csv"):
        import pandas as pd
        %run gesLmUtilsLib.ipynb
#         Lm should always has to be in (N,21,2) shape and reshape to (N,42) whenever needed
        self.rawLm = pd.read_csv(BasicGesturePath).to_numpy().reshape((-1,21,2))
        self.lmForPrediction = self.rawLm.reshape((-1,1,42))
        self.oneRad = np.pi / 180
        self.uniqueGes = self.rawLm.shape[0]
    def show(self):
        showLandmark(self.rawLm)
        
#     Calling from generator will automatically reshape (21,2) into (42)
    def generator(self,BatchSize=64,lmShape=(21,2),rotateRange=5,NoiseRange=5):
        while True:
            yield self.generatePairs(BatchSize,lmShape,rotateRange,NoiseRange,True)
    
    def generatePairs(self,BatchSize,lmShape=(21,2),rotateRange=5,NoiseRange=5,forTraining=False):
        assert type(lmShape) == tuple,"lmShape parameter is expected to be tuple, instead got " + type(lmShape)
        
        firstLm = []
        counterLm = []
        labels = np.zeros(BatchSize,dtype = np.int)
        labels[0:BatchSize//2] = 1
        rotateLmRads = np.random.randint(-rotateRange,rotateRange,(BatchSize*2))*self.oneRad
        
        for i in range(BatchSize):
            gesId = np.random.randint(1,11)
            firstLm.append(self.rawLm[gesId-1])
            if labels[i] == 0:
                counterGesId = (np.random.randint(1,11) + gesId) % gesId
                counterLm.append(self.rawLm[counterGesId-1])
            else:
                counterLm.append(firstLm[i])
        
#         Roatate Landmarks
        firstLm = rotateAll(firstLm,rad=rotateLmRads[0:BatchSize])
        counterLm = rotateAll(counterLm,rad=rotateLmRads[BatchSize:])
        
#         Add Noise
        firstLm = addNoiseSingle(firstLm,NoiseRange)
        counterLm = addNoiseSingle(counterLm,NoiseRange)
        
#         Shuffle the samples and labels
        from sklearn.utils import shuffle
        firstLm, counterLm, labels = shuffle(firstLm, counterLm, labels)
            
#         Why landmarks are not normalized
        if forTraining:
            firstLm = np.reshape(firstLm,(-1,42))
            counterLm = np.reshape(counterLm,(-1,42))
            return ([firstLm,counterLm],labels)
        else:
            EfirstLm = np.expand_dims(firstLm,axis=1)
            EcounterLm = np.expand_dims(counterLm,axis=1)
            res = np.append(EfirstLm,EcounterLm,axis=1)
            return res, labels
        
    def makeOneShotTask(self,N,gesInd=None):
        if gesInd is None:
            gesInd = np.random.randint(0,self.uniqueGes)
        
        gesIndList = np.arange(self.uniqueGes)
        gesIndList = np.delete(gesIndList,gesInd)
        
        subjectGes = self.rawLm[gesInd]
        compGes = [subjectGes]
        
        for i in range(N-1):
            tempInd = np.random.choice(gesIndList)
            compGes.append(self.rawLm[tempInd])
        
        rad = np.random.randint(-10,10,N)*self.oneRad
        radS = np.random.randint(-10,10)*self.oneRad
        
        subjectGes = rotateAll(subjectGes,radS)
        compGes = rotateAll(compGes,rad)
        
        subjectGes = addNoiseSingle(subjectGes,2)
        compGes = addNoiseSingle(compGes,2)
        
        labels = np.zeros(N,dtype=np.int16)
        labels[0] = 1
        from sklearn.utils import shuffle
        compGes, labels = shuffle(compGes, labels)
        subjectGes = np.reshape(subjectGes,(-1,42))
        compGes = np.reshape(compGes,(-1,42))
        compGes = np.expand_dims(compGes, axis=1)
        return subjectGes, compGes, labels
    
    def evaluateModel(self,model,N,iterations=100,gesInd=None):
        i = 0
        for _ in range(iterations):
            sGes,mGes,labels = self.makeOneShotTask(N,gesInd)
            res = []
            for lm in mGes:
                inp = [sGes,lm]
                res.append(model.predict(inp))

            if np.argmax(res) == np.argmax(labels):
                i += 1
        print(i/iterations)
        
    def predict(self,model,landmark):
        res = []
        if landmark.shape == (42,):
            landmark = np.expand_dims(landmark,axis=0)
        for lm in self.lmForPrediction:
            res.append(model.predict([lm,landmark]))
        return np.argmax(res)

In [152]:
ges = Generator2()
# x,l=ges.generatePairs(2,forTraining=True,NoiseRange=2)
# x,y = x
# showLandmark(y)
# ges.predict(sim,y[0])

ges.evaluateModel(sim,8,iterations=1000)

10.0


In [147]:
# Saimese model 

# Specs 
# Input Shape = (N,42)
# output Shape = (1) wheather both landmarks are same or not

#First try use only dense layers
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization, Dropout, Lambda, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.models import Sequential, Model
import tensorflow.keras.backend as K

# Upsampling the features
d1  = 256
dr1 = 0.1
d2  = 512
dr2 = 0.05
d3  = 1024
dr3 = 0.3
d4  = 2048
dr4 = 0.5

def getsim(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    model = Sequential()
    
    model.add(Dense(d1))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(d2))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(d3))
    model.add(Dropout(dr3))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(d4))
    model.add(Dropout(dr4))
    model.add(Activation('sigmoid'))
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    return siamese_net

sim = getsim((42))
BatchSize=32
sim.compile(loss='binary_crossentropy',optimizer='adam')
gObj = Generator2()
Gen = gObj.generator(BatchSize)
# print(np.shape(Gen.__next__()[0]))
sim.fit(Gen,batch_size=BatchSize)
# print(sim.summary())
print("Saimese net generated")

    502/Unknown - 35s 69ms/step - loss: 0.1117

KeyboardInterrupt: 

In [36]:
x,y = gObj.generatePairs(1000,(21,2),forTraining=True)
# print(y)
res=sim.predict(x)
f = lambda a:1 if a > 0.5 else 0
res2 = np.apply_along_axis(f,-1,res)
# print(res2)
print("Difference"+str(abs(sum(y-res2))/100))

Inside for Training
Difference0.08


In [160]:
def train(model,pairGen=None,epochs=100,batchSize=64,NWay=20,Nval=250):
    print("Starting training")
    best = -1
    lossEvery = 10
    evalEvery = 10
    if not pairGen:
        pairGen = pairGenerator(path="garbage/GestureProject/biggest.csv",noOfGes=20,normalize=True)
    for i in range(epochs):
        (inputs,targets) = pairGen.generatePairs(batchSize=64, trainData=True)
        print(np.shape(inputs))
        loss=model.train_on_batch(inputs, targets)
        if i % lossEvery == 0:
            print("Iteration: {} Loss: {}".format(i,loss))
    print("Iteration: {} Loss: {}".format(i,loss))
    
pairGen = pairGenerator("garbage/GestureProject/biggest.csv",noOfGes=20,normalize=True)
train(sim,pairGen,epochs=1000,batchSize=64)

5012
Landmarks is loaded from csv file garbage/GestureProject/biggest.csv with shape of array (100240, 21, 2)
Starting training
(2, 64, 42)


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [172]:
pairGen.evaluate(sim,k=250,reshape=True)

Got 0.728% correct


0.728

In [150]:
i=1

In [151]:
i+=1
sim.save("garbage/GestureProject/models/GesSaimeseModel/iteration"+str(i))

INFO:tensorflow:Assets written to: garbage/GestureProject/models/GesSaimeseModel/iteration2\assets


In [40]:
pairGen.evaluate(load_model("garbage/GestureProject/models/GesSaimeseModel/iteration"+str(i)),k=1000, reshape=True)

NameError: name 'pairGen' is not defined

In [35]:
%run "Gesture Prediction with Saimese.ipynb"

In [13]:
main = pd.read_csv("garbage/GestureProject/NewGes10LR.csv").to_numpy().reshape((-1,42))
mainP = np.expand_dims(preprocessLm(main, trainData=True, normalize=True),axis=1)
sub = pd.read_csv("garbage/GestureProject/TestingCoordinates/newTest1.csv").to_numpy().reshape((-1,42))
subP = np.expand_dims(preprocessLm(sub, trainData=True, normalize=True),axis=1)
showLandmark(mainP.squeeze())
showLandmark(subP.squeeze())
# model = load_model('garbage/GestureProject/models/gestureLm Saimese/iteration43')

[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)
[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)
[showLandmark]: Noramlized Landmarks are Passed
[showLandmark] (20, 21, 2)
Array
[showLandmark]: Noramlized Landmarks are Passed
[showLandmark] (10, 21, 2)
Array


In [14]:
model = load_model('garbage/GestureProject/models/gestureLm Saimese/iteration44')

In [101]:
modelPath="garbage/GestureProject/models/GesSaimeseModel/iteration0"
subPath="garbage/GestureProject/TestingCoordinates/newTest1.csv"
pred = PredictLandmarks(ModelPath=modelPath,SubjectGesPath=subPath,trainData=True)
print(pred.predictAll())

[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)
[normalizeLm]: Got landmarks of shape 42. Reshaping to (21,2)
[0, 2, 6, 6, 8, 10, 16, 15, 3, 18]


In [2]:
%run gesLmUtilsLib.ipynb

In [18]:
pg = pairGenerator(path="garbage/GestureProject/NewGes10LRTrainData.csv",noOfGes=20,relocate=False)
ost = pg.makeOneShotTask(N=8)
genPair = pg.generatePairs(batchSize=8)
print(genPair[1])
showLandmark(genPair[0])

2002
Landmarks is loaded from csv file garbage/GestureProject/NewGes10LRTrainData.csv with shape of array (40040, 21, 2)
[1 0 1 1 0 0 0 1]
[showLandmark]: Noramlized Landmarks are Passed
[showLandmark] (2, 8, 21, 2)
Pairs


In [92]:
showLandmark(preprocessLm(pd.read_csv("garbage/GestureProject/NewGes10LR.csv").to_numpy().reshape((-1,21,2)), normalize=True,trainData=True))

[showLandmark]: Noramlized Landmarks are Passed
[showLandmark] (20, 21, 2)
Array


In [43]:
#Shuffling landmarks class-wise

import pandas as pd

lms=pd.read_csv("garbage/GestureProject/NewGes10LRTrainData.csv").to_numpy().reshape((-1,42))
uges = 10
spg = int(lms.shape[0]/uges)
lmsDir = {}

for i in range(uges):
    lmsDir[i] = lms[i*spg:(i+1)*spg]
    
from sklearn.utils import shuffle

for i in range(uges):
    lmsDir[i] = shuffle(lmsDir[i])

for i in range(uges):
    showLandmark(lmsDir[i][0::400])

shuffled = []

for i in range(uges):
    shuffled.append(lmsDir[i])
print(np.shape(shuffled))

shuffled = np.reshape(shuffled,(-1,42))
print(type(shuffled))
pd.DataFrame(shuffled).to_csv("garbage/GestureProject/FeedGes.csv",index=False)

In [9]:
import json

labels = {
    0:"LeftOne",
    1:"RightOne",
    2:"LeftTwo",
    3:"RightTwo",
    4:"LeftThree",
    5:"RightThree",
    6:"LeftFour",
    7:"RightFour",
    8:"LeftFive",
    9:"RightFive",
    10:"LeftTasty",
    11:"RightTasty",
    12:"LeftSpiderMan",
    13:"RightSpiderMan",
    14:"LeftSuperMan",
    15:"RightSuperMan",
    16:"LeftThumbsUp",
    17:"RightThumbsUp",
    18:"RightThumbsDown",
    19:"LeftThumbsDown"
}

js = json.dumps(labels)

with open("garbage/GestureProject/GestureLabels.txt","w") as file:
    file.write(js)

In [46]:
model = load_model("garbage/GestureProject/models/gestureLm Saimese/iteration45")
mainP = np.expand_dims(preprocessLm(pd.read_csv("garbage/GestureProject/NewGes10LR.csv").to_numpy().reshape((-1,21,2)),trainData=True,normalize=True),1)
subP = np.expand_dims(preprocessLm(pd.read_csv("garbage/GestureProject/TestingCoordinates/newTest1.csv").to_numpy().reshape((-1,21,2)),trainData=True,normalize=True),1)
# showLandmark(mainP.squeeze())
# showLandmark(subP.squeeze()[0])

In [201]:
mres=[]
for i in range(10):
    res = []
    for lm in mainP:
        res.append(sim.predict([lm,subP[i]]))
    mres.append(np.argmax(res))
print(mres)

[17, 6, 2, 2, 4, 2, 2, 0, 18, 19]


In [200]:
res = []
for lm in mainP:
    res.append(sim.predict([lm,subP[3]]))
print(np.asarray(res).squeeze())

[0.05557037 0.1221828  0.7894304  0.29697892 0.6777727  0.24764109
 0.62301165 0.20701557 0.7260142  0.32250762 0.7401893  0.2893918
 0.2468455  0.36953175 0.31682247 0.33255556 0.64207524 0.43353727
 0.6812948  0.00365604]


In [ ]:
import pandas as pd

images = pd.read_csv("garbage/GestureProject/RawGestures/")

In [138]:
ge = Generator("garbage/GestureProject/RawGestures/RelocatedRawGes10R.csv")
lmShape = (21,2)
x,y = ge.generatePairs(8,lmShape,rotateRange=10,NoiseRange=5,forTraining=True)
print(y)
# showLandmark(x)

[0 1 0 0 1 1 0 1]


In [141]:
x[0][2]

array([ 81.7075859 , 188.36319959,  56.74686217, 183.88604228,
        38.09353689, 160.44792524,  58.85552864, 124.18332114,
        81.98246954, 118.41022897,  37.2504142 , 117.22596532,
        21.13961099,  79.03410284,   3.7920841 ,  57.05986616,
        -6.30353308,  27.25259158,  69.30637087, 104.61425739,
        64.58112707,  65.88274291,  61.1977037 ,  27.61097952,
        67.50574825,  -2.17155625,  94.78835729, 102.87362711,
        99.28951689,  62.57286184, 100.56579893,  36.2036453 ,
       109.53777774,  15.54802149, 117.51452884, 112.84143033,
       123.04164695,  88.2757912 , 136.77592572,  67.27078242,
       141.29231132,  39.93400045])

In [85]:
%run gesLmUtilsLib.ipynb
x=pd.read_csv("garbage/GestureProject/RawGestures/RawGesture10R.csv").to_numpy().reshape((-1,21,2))
rlm=[]
oneRad = np.pi/180
axis=0
rads=np.random.randint(-8,8,x.shape[0])*np.pi/180

rlm = rotateAll(x,rads)
rlm = np.expand_dims(np.asarray(rlm),axis=axis)
x=np.expand_dims(x, axis=axis)
inp = np.append(x,rlm, axis=axis)

In [43]:
import json

labelsPath = "garbage/GestureProject/GesLabels.txt"
with open(labelsPath,'r') as file:
    file = json.load(file)
lastInd = int(list(file.keys())[-1])
gesName = input("Write name of gesture")
file[lastInd+1]=gesName
with open(labelsPath,'w') as newFile:
    json.dump(file,newFile)

Write name of gesturenew Gesname


In [95]:
import pandas as pd
%run gesLmUtilsLib.ipynb
im = pd.read_csv("garbage/GestureProject/TestingCoordinates/preprossed10RLTesting.csv").to_numpy()
showLandmark(im)
# import cv2 as cv
# import numpy as np
# back = np.zeros((480,640,3))
# back = impersonateLm(back, im[0]*(480,640))
# cv.imshow("ogg",back)
# key = cv.waitKey(0)
# cv.destroyAllWindows()
# print(key)

[showLandmark]: Noramlized Landmarks are Passed
[showLandmark] (21, 21, 2)
Array
